# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? Does CLT apply?

For this test, we will be doing proportion hypothesis testing.  We want to test whether race as any significant impact on the callback received.  If there is no impact then %callback for 'w' will be equal to %callback for 'b'.
But we don't believe that this is the case, so we can set %callback for 'w' to be the known proportion and %callback for 'b' to be the observed proportion.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

For CLT to apply in this case, the data needs to satisfy three conditions.  

1. Random. 

We can say that these data were picked without any predetermined order.

2. Normal.

Good way to calculate this is :  np >= 10 and n(1-p) >= 10 .  

Our sample size is 4870 so p = 235/4870 = 0.048.

In either way, both numbers are way above 10.

3. Independent.

Good way to determine this is to check if our sample size does not exceed 10% of the population size.

Our population is the human population so we can safely way the sample is independent.


CLT conditions are met.

## 2. What are the null and alternate hypotheses?

Ho : p = 0.048

Ha : p < 0.048

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [7]:
#our population proportion 
p = 0.048

n = 4870


#this is observed proportion
p_obs = 157/4870

#this is std of sampling distribution of sample proportion
sample_std = math.sqrt((p*(1-p))/n)

#we will first calculate the z stat
z = (p_obs - p) / sample_std

print(p_obs)
print(sample_std)
print('z stat is : ' +str(z))

0.03223819301848049
0.0030631947730529866
z stat is : -5.14554514135914


In [8]:
#margin of error is calculated by critical value * standard error
#for proportion test, margin of error is just critical value * sample_std

se = sample_std / math.sqrt(n)

me = abs(z * se)

print(me)

0.00022586114436624687


Our confidence interval would be our proportion +- margin of error.

In [9]:
p_value = stats.binom_test(157, n=4870, p=0.048, alternative = 'less')
print(p_value)

3.15393688576118e-08


## 4. Write a story describing the statistical significance in the context or the original problem.

When identical resumes were sent into hiring companies with the only difference being race, our sample showed 0.048% callback for 'w' and 0.032% for 'b'.  It is highly unlikely that there is no profiling difference for racial reasons as we got a very low p-value.  

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis conducted simply infers the difference in proportion of callback success between two races.  This does not show in any way that a race type is the most important factor in callback success.  